In [1]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [2]:
from sklearn.metrics import roc_auc_score, accuracy_score

import torch
import torch.nn as nn
from torch.utils import data

import importlib.util
import sys

def load_module_from_path(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

loaded_module = load_module_from_path('model', '/kaggle/input/model/model.py')

from model import Transformer, KAN, TransformerKAN
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from tqdm import tqdm

def load_data(data_array, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_array)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

def train(model, x_train, y_train, x_valid, y_valid, num_epochs, lr, batch_size, weight_decay):
    best_model = model
    best_validation = 0
    train_iter = load_data((x_train, y_train), batch_size)
    val_iter = load_data((x_valid, y_valid), batch_size, is_train=False)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
    loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(5).to(device))
    
    for epoch in range(num_epochs):
        model.train()
        print("training")
        for X, y in tqdm(train_iter):
            logits = model(X)
            l = loss(logits, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        model.eval()
        y_prob = np.array([])
        y_pred = np.array([])
        cur_validation = 0
        print("validation")
        with torch.no_grad():
            for X, y in tqdm(val_iter):
                m = model.predict(X)
                pred, prob = m[0].flatten(), m[1].flatten()
                y_prob = np.concatenate((y_prob, prob.cpu().detach().numpy()))
                y_pred = np.concatenate((y_pred, pred.cpu().detach().numpy()))
            cur_validation = validation(y_valid, y_prob)
            print(f"roc_auc_score: {cur_validation}")
            print(f"accuracy: {accuracy(y_valid, y_prob)}")
            if cur_validation > best_validation:
                best_validation = cur_validation
                best_model = model
                print("Save new model")
                torch.save(model, PATH)
        # scheduler.step()
    return y_prob, y_pred, best_model

def validation(y_valid, y_score):
    return roc_auc_score(y_true=y_valid, y_score=y_score)

# only use to get test batch
def get_batch(x, batch_size):
    ix = torch.arange(0, len(x), batch_size)
    return [x[i:i+batch_size] for i in ix]

def accuracy(y_true, y_pred):
    y_pred = np.where(y_pred > 0.5, 1., 0.)
    return accuracy_score(y_true=y_true, y_pred=y_pred)

def predict(model, x_test, batch_size):
    test_iter = get_batch(x_test, batch_size)
    res = np.array([])
    model.eval()
    with torch.no_grad():
        for X in test_iter:
            X = X.to(device)  # 確保數據位於相同設備
            label, prob = model.predict(X)
            res = np.concatenate((res, prob.cpu().flatten().numpy()))  # 將結果移回 CPU 並存入 res
            
    ret = res[:len(x_test)] # 確保輸出與輸入長度相同
#     print(ret.shape)
    return ret

In [4]:
class Pipeline:
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df


class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return expr_max +expr_last+expr_mean
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return  expr_max +expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

In [6]:
%%time
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
    ]
}

CPU times: user 4min 49s, sys: 1min 37s, total: 6min 27s
Wall time: 2min 17s


In [7]:
%%time
df_train = feature_eng(**data_store)
print("train data shape:\t", df_train.shape)
del data_store
gc.collect()
df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
print("train data shape:\t", df_train.shape)
nums=df_train.select_dtypes(exclude='category').columns
from itertools import combinations, permutations
#df_train=df_train[nums]
nans_df = df_train[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
            #print(str(gg)+'-'+str(n),', ',end='')
        use.append(vx)
        #print()
    print('Use these',use)
    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    # 计算列之间的相关性
    correlation_matrix = matrix.corr()

    # 分组列
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
    return groups

uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+list(df_train.select_dtypes(include='category').columns)
print(len(uses))
df_train=df_train[uses]

train data shape:	 (1526659, 861)
Memory usage of dataframe is 4322.75 MB
Memory usage after optimization is: 1528.81 MB
Decreased by 64.6%
train data shape:	 (1526659, 472)
Use these ['case_id', 'WEEK_NUM', 'target', 'month_decision', 'weekday_decision', 'credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'isbidproduct_1095L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L', 'max_mainoccupationinc_384A', 'max_birth_259D

In [8]:
sample = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv")
device='gpu'
#n_samples=200000
n_est=6000
DRY_RUN = True if sample.shape[0] == 10 else False   
if DRY_RUN:
    device='cpu'
    df_train = df_train.iloc[:50000]
    #n_samples=10000
    n_est=600
print(device)

cpu


In [9]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_file(TEST_DIR / "test_applprev_2.parquet", 2),
        read_file(TEST_DIR / "test_person_2.parquet", 2)
    ]
}

In [10]:
df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)
del data_store
gc.collect()
df_test = df_test.select([col for col in df_train.columns if col != "target"])
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

gc.collect()

test data shape:	 (10, 860)
train data shape:	 (50000, 389)
test data shape:	 (10, 388)
Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.02 MB
Decreased by 40.3%


0

### Feature Selection

In [11]:
y = df_train["target"]
weeks = df_train["WEEK_NUM"]
df_train= df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

test_id = df_test["case_id"]
test_week = df_test["WEEK_NUM"]

df_test = df_test.drop(columns=["case_id", "WEEK_NUM"])


In [12]:
df_train[cat_cols] = df_train[cat_cols].astype(str)
df_test[cat_cols] = df_test[cat_cols].astype(str)

In [13]:
from sklearn.preprocessing import LabelEncoder

df_all = pd.concat([df_train, df_test], axis=0)

# handle na value in numeric feature for dl
numeric_features = df_all.select_dtypes(include=np.number).columns
df_all[numeric_features] = df_all[numeric_features].apply(
    lambda x: (x-x.mean())/(x.std()))
df_all[numeric_features] = df_all[numeric_features].fillna(0)

types = df_all.dtypes
cat_features = [col for col in df_all.columns if df_all[col].dtype.name == 'category' or df_all[col].dtype.name == 'object']

for col in df_all.columns:
    if types[col] == 'object' or types[col] == 'category':
#         print(col, df_all[col].nunique())
        l_enc = LabelEncoder()
        df_all[col] = l_enc.fit_transform(df_all[col].values)


In [14]:
df_all.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all = df_all.dropna(axis="columns")

df_train_d = df_all.head(len(df_train))
df_test_d = df_all.tail(len(df_test))

del df_all
gc.collect()

0

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_test_d = torch.tensor(df_test_d.values, dtype=torch.float32).to(device)

In [16]:
num_epoch, lr, batch_size, weight_decay = 5, 1e-3, 256, 1e-4

def get_model(model_name, x_train_d, device, batch_size):
    # "Transformer", "KAN", "TransformerKAN"
    if model_name == "Transformer":
        print("Train Transformer")
        model = Transformer(in_features=len(x_train_d[0]), drop=0.).to(device)
    elif model_name == "KAN":
        print("Train KAN")
        model = KAN([len(x_train_d[0]), batch_size, 1]).to(device)
    elif model_name == "TransformerKAN":
        print("Train TransformerKAN")
        model = TransformerKAN(in_features=len(x_train_d[0]), drop=0.).to(device)
    return model

In [17]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score, auc

import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
import catboost as cat
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from xgboost import XGBClassifier
import torch
import torch.nn as nn
from torch.utils import data

In [18]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,  
    "learning_rate": 0.05,
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "device": "gpu", 
    "verbose": 100,
}

In [19]:
%%time

fitted_models_cat = []
fitted_models_lgb = []
fitted_models_xgb = []
fitted_models_transformer = []
fitted_models_kan = []
fitted_models_transformerkan = []

cv_scores_cat = []
cv_scores_lgb = []
cv_scores_xgb = []
cv_scores_transformer = []
cv_scores_kan = []
cv_scores_transformerkan = []

for idx_train, idx_valid in cv.split(df_train, y, groups=weeks):
    X_train, y_train = df_train.iloc[idx_train], y.iloc[idx_train] 
    X_valid, y_valid = df_train.iloc[idx_valid], y.iloc[idx_valid]
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid, cat_features=cat_cols)
    clf = CatBoostClassifier(
        eval_metric='AUC',
        task_type='GPU',
        learning_rate=0.03,
        iterations=n_est,
        random_seed=3107)
    
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    fitted_models_cat.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_cat.append(auc_score)
    
    
    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")
    
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)])
    
    fitted_models_lgb.append(model)
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_lgb.append(auc_score)
    
    xgb_model = XGBClassifier(
        device="cuda",
        objective='binary:logistic',
        tree_method="hist",
        enable_categorical=True,
        eval_metric='auc',
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=1,
        max_depth=20,
        n_estimators=1200,
        random_state=42,
    )

    # Training the model on the training data
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=True,
    )
    fitted_models_xgb.append(xgb_model)
    y_pred_valid = xgb_model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_xgb.append(auc_score)
    
print("CV AUC scores: ", cv_scores_cat)
print("Maximum CV AUC score: ", max(cv_scores_cat))

print("CV AUC scores: ", cv_scores_lgb)
print("Maximum CV AUC score: ", max(cv_scores_lgb))

print("CV AUC scores: ", cv_scores_xgb)
print("Maximum CV AUC score: ", max(cv_scores_xgb))

del df_train
gc.collect()

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5607415	best: 0.5607415 (0)	total: 250ms	remaining: 2m 29s
300:	test: 0.7348446	best: 0.7348446 (300)	total: 27.6s	remaining: 27.4s
599:	test: 0.7384720	best: 0.7385753 (590)	total: 54.9s	remaining: 0us
bestTest = 0.7385753095
bestIteration = 590
Shrink model to first 591 iterations.
[LightGBM] [Info] Number of positive: 1397, number of negative: 38016
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.905121
[LightGBM] [Info] Total Bins 29252
[LightGBM] [Info] Number of data points in the train set: 39413, number of used features: 371
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 187 dense feature groups (7.07 MB) transferred to GPU in 0.006453 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035445 -> initscore=-3.303680
[LightGBM] [Info] Start training from score -3.303680
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
Training until validation scores don't improve for 100 rounds
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 64 and depth = 10
[

Default metric period is 5 because AUC is/are not implemented for GPU


...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6063113	best: 0.6063113 (0)	total: 141ms	remaining: 1m 24s
300:	test: 0.7177612	best: 0.7177612 (300)	total: 28.1s	remaining: 27.9s
599:	test: 0.7263001	best: 0.7264063 (585)	total: 56.3s	remaining: 0us
bestTest = 0.7264062762
bestIteration = 585
Shrink model to first 586 iterations.
[LightGBM] [Info] Number of positive: 1398, number of negative: 38807
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.903913
[LightGBM] [Info] Total Bins 29212
[LightGBM] [Info] Number of data points in the train set: 40205, number of used features: 373
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 185 dense feature groups (7.21 MB) transferred to GPU in 0.005900 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScor

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6155595	best: 0.6155595 (0)	total: 143ms	remaining: 1m 25s
300:	test: 0.7249786	best: 0.7249786 (300)	total: 28.4s	remaining: 28.3s
599:	test: 0.7302017	best: 0.7303635 (585)	total: 56.4s	remaining: 0us
bestTest = 0.7303634882
bestIteration = 585
Shrink model to first 586 iterations.
[LightGBM] [Info] Number of positive: 1402, number of negative: 38397
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.900021
[LightGBM] [Info] Total Bins 29275
[LightGBM] [Info] Number of data points in the train set: 39799, number of used features: 369
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 185 dense feature groups (7.14 MB) transferred to GPU in 0.005928 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScor

15394

In [20]:
for idx_train, idx_valid in cv.split(df_train_d, y, groups=weeks):
    #  DL part
    x_train_d, y_train_d = df_train_d.iloc[idx_train], y.iloc[idx_train]
    x_valid_d, y_valid_d = df_train_d.iloc[idx_valid], y.iloc[idx_valid]
    
    x_train_d  = torch.tensor(x_train_d.values, dtype=torch.float32).to(device)
    x_valid_d  = torch.tensor(x_valid_d.values, dtype=torch.float32).to(device)
    y_train_d  = torch.tensor(y_train_d.values, dtype=torch.float32).to(device).reshape(-1, 1)
    y_valid_d  = torch.tensor(y_valid_d.values, dtype=torch.float32).reshape(-1, 1)
    
    print(x_train_d.shape)
    print(y_train_d.shape)
    print(x_valid_d.shape)
    print(y_valid_d.shape)
    
    transformer_model = get_model('Transformer', x_train_d, device, batch_size)
    PATH = f"/kaggle/working/Transformer.pth"
#     prob, pred = train(transformer_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    prob, pred, transformer_model = train(transformer_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    
    kan_model = get_model('KAN', x_train_d, device, batch_size)
    PATH = f"/kaggle/working/KAN.pth"
#     prob, pred = train(kan_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    prob, pred, kan_model = train(kan_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    
    transformerKan_model = get_model('TransformerKAN', x_train_d, device, batch_size)
    PATH = f"/kaggle/working/TransformerKAN.pth"
#     prob, pred = train(transformerKan_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    prob, pred, transformerKan_model = train(transformerKan_model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
    
    fitted_models_transformer.append(transformer_model)
    fitted_models_kan.append(kan_model)
    fitted_models_transformerkan.append(transformerKan_model)
    
# print("CV AUC scores: ", cv_scores_transformer)
# print("Maximum CV AUC score: ", max(cv_scores_transformer))

# print("CV AUC scores: ", cv_scores_kan)
# print("Maximum CV AUC score: ", max(cv_scores_kan))

# print("CV AUC scores: ", cv_scores_transformerkan)
# print("Maximum CV AUC score: ", max(cv_scores_transformerkan))

torch.Size([39413, 347])
torch.Size([39413, 1])
torch.Size([10587, 347])
torch.Size([10587, 1])
Train Transformer
training


100%|██████████| 154/154 [00:03<00:00, 49.47it/s]


validation


100%|██████████| 42/42 [00:00<00:00, 91.99it/s] 


roc_auc_score: 0.6307832395341632
accuracy: 0.967318409369982
Save new model
training


...

roc_auc_score: 0.6810416208145711
accuracy: 0.966571904715224
training


100%|██████████| 156/156 [00:07<00:00, 20.89it/s]


validation


100%|██████████| 40/40 [00:00<00:00, 53.86it/s]


roc_auc_score: 0.6956172336464164
accuracy: 0.966571904715224
Save new model


In [21]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
#     def predict(self, X):
#         y_preds = [estimator.predict(X) for estimator in self.estimators]
#         return np.mean(y_preds, axis=0)
    
#     def predict_proba(self, X):
#         y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
#         return np.mean(y_preds, axis=0)
    
    def predict_proba_with_dl(self, X1, X2):
        batch_size = 256
        while len(X2)%batch_size == 1:
            batch_size += 32
        y_preds = [estimator.predict_proba(X1)[:,1] for estimator in self.estimators[:5]]
        
        X1[cat_cols] = X1[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X1)[:,1] for estimator in self.estimators[5:15]]
        y_preds += [predict(estimator, X2, batch_size) for estimator in self.estimators[15:]]
        
        y_preds = np.asarray(y_preds)
#         print(y_preds.shape)
#         print(y_preds)
        return np.mean(y_preds, axis=0)
    

model = VotingModel(fitted_models_cat+fitted_models_lgb+fitted_models_xgb+\
                    fitted_models_transformer+fitted_models_kan+fitted_models_transformerkan)

# Submision

In [22]:
# df_test = df_test.drop(columns=["WEEK_NUM"])
# df_test = df_test.set_index("case_id")
# test_id = df_test["case_id"]
# test_week = df_test["WEEK_NUM"]

y_pred = pd.Series(model.predict_proba_with_dl(df_test, x_test_d), index=df_test.index)
# print(y_pred)
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm["score"] = y_pred
df_subm["case_id"] = test_id
df_subm = df_subm.set_index("case_id")

df_subm.to_csv("submission.csv")
df_subm

,score
case_id,
57543,0.011780
57549,0.018310
57551,0.029097
57552,0.020751
57569,0.030649
57630,0.070932
57631,0.083569
57632,0.033106
57633,0.036539
